In [1]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")

  Activating project at `D:\Science\Code\Julia\envr`


In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, JLD

In [2]:
function TM(u, p, t)

    U(y) = U0 + ΔU0 / ( 1 + exp( -50*(y-ythr) ) )
    σ(x) = 1 / ( 1 + exp( -20*(x-xthr) ) )
    
    # u[1] - E; u[2] - x; u[3] - y
    # α - 1, τ - 2, τD - 3, τy - 4, J - 5, xthr - 6, ythr - 7, U0 - 8, ΔU0 - 9, β - 10, I0 -11

    du1 = (-u[1] + p[1] * log( 1 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1-u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10]*σ(u[2])
    return SA[du1, du2, du3]
end;

In [3]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [4]:
t = 1000.0; tt = 250.0; tstep = 0.001
time_attract = 100.0
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [5]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [6]:
u0 = SA[17.6055, 0.6181, 0.429232]

3-element SVector{3, Float64} with indices SOneTo(3):
 17.6055
  0.6181
  0.429232

In [7]:
I0range = range( -1.40, -1.709, length = 200 )
U0range = range( 0.3,  0.47, length = 200)

0.3:0.0008542713567839196:0.47

In [8]:
Λs = zeros(length(I0range), length(U0range), 3)
u0s = zeros(length(I0range), length(U0range), 3);

In [9]:
filenameLS = "LS_200x200.jld"
filenameu0 = "u0s_200x200.jld";

In [10]:
function init_ds(U0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0range[1]]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function init_ds(U0_, I0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function goto_attractor(ds_, time_attract)
    tr = trajectory(ds_, time_attract; Ttr = tt, diffeq = integ_set)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t; diffeq = integ_set)
    return ΛΛ
end
function save_output(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end
function save_output(index_I0, idx_U0, ΛΛ, u0_lc)
    Λs[index_I0, idx_U0, :] = ΛΛ
    u0s[index_I0, idx_U0, :] = u0_lc
end
function save_tofile()
    save(filenameLS, "data", Λs)
    save(filenameu0, "data", u0s)
end

save_tofile (generic function with 1 method)

In [11]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        global u0_lc = u0
    end
    
    output(idx_U0, U0_, u0_lc) # Вывод
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ΛΛ = spectrum(ds) # Вычисление спектра
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    output_end_iter(ΛΛ, u0_lc) # Вывод
    save_tofile()
    separate()
    
end

Индекс U0: 1; U0: 0.3
Начальные условия: [17.6055, 0.6181, 0.429232]
Спектр: [0.003639745227994901, -2.7690126257195735, -2.778961472803736]
Последняя точка: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]

---------
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [0.0028122157682933286, -2.769018726037493, -2.778087894509237]
Последняя точка: [11.804131991116028, 0.7625654109056821, 0.433069419058698]

---------
Индекс U0: 3; U0: 0.30170854271356784
Начальные условия: [11.804131991116028, 0.7625654109056821, 0.433069419058698]
Спектр: [0.004773259669182011, -2.769850351626659, -2.778629499090303]
Последняя точка: [7.197378753952849, 0.8113339141768326, 0.4305100369666299]

---------
Индекс U0: 4; U0: 0.30256281407035174
Начальные условия: [7.197378753952849, 0.8113339141768326, 0.4305100369666299]
Спектр: [0.0033644170121265816, -2.7673771788714046, -2.7787347110098892]
Последняя точка: [4.98066

Последняя точка: [4.172769558495282, 0.8134772553410761, 0.42187399666737224]

---------
Индекс U0: 31; U0: 0.3256281407035176
Начальные условия: [4.172769558495282, 0.8134772553410761, 0.42187399666737224]
Спектр: [0.006507379630284479, -2.771131079820452, -2.77716338933884]
Последняя точка: [4.3346530278264055, 0.7897373322789215, 0.4186697806962704]

---------
Индекс U0: 32; U0: 0.3264824120603015
Начальные условия: [4.3346530278264055, 0.7897373322789215, 0.4186697806962704]
Спектр: [0.005126288074613938, -2.770475848364959, -2.7768073267049274]
Последняя точка: [5.0138810552717725, 0.7565292580118121, 0.41669731573383206]

---------
Индекс U0: 33; U0: 0.32733668341708544
Начальные условия: [5.0138810552717725, 0.7565292580118121, 0.41669731573383206]
Спектр: [0.004251527970392027, -2.7705477651284487, -2.77631110990847]
Последняя точка: [6.21630506235756, 0.716209058428229, 0.41654679602537553]

---------
Индекс U0: 34; U0: 0.32819095477386934
Начальные условия: [6.21630506235756,

Спектр: [0.004734819912131501, -2.772166683824316, -2.7759507706538993]
Последняя точка: [11.494435167150874, 0.6215806614263748, 0.4222926594283157]

---------
Индекс U0: 61; U0: 0.3512562814070352
Начальные условия: [11.494435167150874, 0.6215806614263748, 0.4222926594283157]
Спектр: [0.004849797730725268, -2.7705686253601662, -2.7780132102735298]
Последняя точка: [15.116569830629588, 0.6028451082609325, 0.42606292044508076]

---------
Индекс U0: 62; U0: 0.3521105527638191
Начальные условия: [15.116569830629588, 0.6028451082609325, 0.42606292044508076]
Спектр: [0.00296389724473273, -2.7678412120295124, -2.779226148890584]
Последняя точка: [17.861454749915698, 0.6258325378966532, 0.42986690955586054]

---------
Индекс U0: 63; U0: 0.35296482412060304
Начальные условия: [17.861454749915698, 0.6258325378966532, 0.42986690955586054]
Спектр: [0.003848064450546598, -2.7681764363724475, -2.7800312779695173]
Последняя точка: [16.224867450040172, 0.6966558494887568, 0.4328209671257617]

------


---------
Индекс U0: 90; U0: 0.37603015075376883
Начальные условия: [9.47474470808507, 0.7891464594245106, 0.432209017927633]
Спектр: [0.004954115543177585, -2.769192564430658, -2.779063072519909]
Последняя точка: [5.965931897170935, 0.820897958330707, 0.42890827129953696]

---------
Индекс U0: 91; U0: 0.3768844221105528
Начальные условия: [5.965931897170935, 0.820897958330707, 0.42890827129953696]
Спектр: [0.00407432913522878, -2.769508982289695, -2.776906911522677]
Последняя точка: [4.525442213198067, 0.8240123448764632, 0.4250077283202686]

---------
Индекс U0: 92; U0: 0.3777386934673367
Начальные условия: [4.525442213198067, 0.8240123448764632, 0.4250077283202686]
Спектр: [0.005699940194350021, -2.76948559816787, -2.77799723347065]
Последняя точка: [4.160577041918381, 0.810158944187375, 0.42126188913778917]

---------
Индекс U0: 93; U0: 0.3785929648241206
Начальные условия: [4.160577041918381, 0.810158944187375, 0.42126188913778917]
Спектр: [0.005918830751725662, -2.7703721806699,

Спектр: [0.005339437591435539, -2.7703819403396417, -2.777318898991659]
Последняя точка: [5.4771770546028185, 0.7394387663292838, 0.4163843687156469]

---------
Индекс U0: 120; U0: 0.40165829145728643
Начальные условия: [5.4771770546028185, 0.7394387663292838, 0.4163843687156469]
Спектр: [0.003252129079884735, -2.7687236585485198, -2.7773100044085757]
Последняя точка: [6.9328548390289075, 0.6969493128451736, 0.4171061746018187]

---------
Индекс U0: 121; U0: 0.40251256281407033
Начальные условия: [6.9328548390289075, 0.6969493128451736, 0.4171061746018187]
Спектр: [0.004102024732289125, -2.769849174325084, -2.7773726104592327]
Последняя точка: [9.068890310994433, 0.6530464167304493, 0.4195178598802596]

---------
Индекс U0: 122; U0: 0.4033668341708543
Начальные условия: [9.068890310994433, 0.6530464167304493, 0.4195178598802596]
Спектр: [0.004315200773150403, -2.7715942393800113, -2.7761606309929108]
Последняя точка: [12.080078966680182, 0.6163580764257998, 0.4229221261466773]

-------


---------
Индекс U0: 149; U0: 0.4264321608040201
Начальные условия: [16.63377803829136, 0.6066482110226946, 0.4277678033549307]
Спектр: [0.002825020261159952, -2.7672993361975524, -2.779770200363191]
Последняя точка: [17.831159021912942, 0.6525489855685237, 0.43137956506988046]

---------
Индекс U0: 150; U0: 0.42728643216080403
Начальные условия: [17.831159021912942, 0.6525489855685237, 0.43137956506988046]
Спектр: [0.004327632766908972, -2.7691879902538594, -2.7795078417289103]
Последняя точка: [13.995621056620646, 0.733510916604973, 0.433283180389627]

---------
Индекс U0: 151; U0: 0.42814070351758793
Начальные условия: [13.995621056620646, 0.733510916604973, 0.433283180389627]
Спектр: [0.004582854391569462, -2.7727608871966285, -2.7758200919228893]
Последняя точка: [8.68203850423881, 0.7973374088567176, 0.4317398219600641]

---------
Индекс U0: 152; U0: 0.4289949748743719
Начальные условия: [8.68203850423881, 0.7973374088567176, 0.4317398219600641]
Спектр: [0.004833668673197389, -2

Спектр: [0.0059928977924917585, -2.77113144511485, -2.7768810023246617]
Последняя точка: [4.270669098662722, 0.8195677449129073, 0.4233027549856462]

---------
Индекс U0: 179; U0: 0.4520603015075377
Начальные условия: [4.270669098662722, 0.8195677449129073, 0.4233027549856462]
Спектр: [0.006544255461109086, -2.7709390415424635, -2.777342913808747]
Последняя точка: [4.207907462066781, 0.8001433024131198, 0.4197970022266744]

---------
Индекс U0: 180; U0: 0.45291457286432163
Начальные условия: [4.207907462066781, 0.8001433024131198, 0.4197970022266744]
Спектр: [0.005831339405340344, -2.7705689559870708, -2.7772500328392358]
Последняя точка: [4.692001936911051, 0.7703233622806498, 0.41726924680407246]

---------
Индекс U0: 181; U0: 0.4537688442211055
Начальные условия: [4.692001936911051, 0.7703233622806498, 0.41726924680407246]
Спектр: [0.005071371002198628, -2.7704564861782983, -2.7770548402137787]
Последняя точка: [5.686753918382361, 0.7324308378590757, 0.4163674449438249]

---------
И

In [ ]:
for (idx_U0, U0_) in enumerate(U0range)
    for (idx_I0, I0_) in enumerate(I0range)
        
        if idx_I0 == 1
            continue
        end
        
        u0_lc = u0s[idx_I0 - 1, idx_U0, :]
        output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        ds = init_ds(U0_, I0_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ΛΛ = spectrum(ds) # Вычисление спектра
        save_output(idx_I0, idx_U0, ΛΛ, u0_lc)
        output_end_iter(ΛΛ, u0_lc)
        save_tofile()
        separate()
    end
end

Индекс I0: 2; I0: -1.4015527638190954
Индекс U0: 1; U0: 0.3
Начальные условия: [16.898370136765177, 0.6822784289462809, 0.4324577396837501]
Спектр: [0.0028529857895003416, -2.762033080490801, -2.7679832492053253]
Последняя точка: [5.012430913861254, 0.75647574170399, 0.41674520628660566]

---------
Индекс I0: 3; I0: -1.403105527638191
Индекс U0: 1; U0: 0.3
Начальные условия: [5.012430913861254, 0.75647574170399, 0.41674520628660566]
Спектр: [0.006333902142119184, -2.7548165548849783, -2.7623025269355512]
Последняя точка: [4.414333718336842, 0.7843633597217637, 0.4182981556925739]

---------
Индекс I0: 4; I0: -1.4046582914572865
Индекс U0: 1; U0: 0.3
Начальные условия: [4.414333718336842, 0.7843633597217637, 0.4182981556925739]
Спектр: [0.0064047641759806115, -2.746949763631592, -2.753429115880218]
Последняя точка: [12.772185607401552, 0.6107337173548409, 0.4238256429999211]

---------
Индекс I0: 5; I0: -1.406211055276382
Индекс U0: 1; U0: 0.3
Начальные условия: [12.772185607401552, 0.6

Спектр: [0.003700664792144052, -1.382492413131204, -3.7112614674456]
Последняя точка: [11.38251654241101, 0.6183389693111019, 0.4238575100856383]

---------
Индекс I0: 30; I0: -1.4450301507537688
Индекс U0: 1; U0: 0.3
Начальные условия: [11.38251654241101, 0.6183389693111019, 0.4238575100856383]
Спектр: [0.005910329470424675, -1.3246821215128763, -3.753715466080889]
Последняя точка: [4.493474351367417, 0.8276414503410922, 0.42744948635019786]

---------
Индекс I0: 31; I0: -1.4465829145728644
Индекс U0: 1; U0: 0.3
Начальные условия: [4.493474351367417, 0.8276414503410922, 0.42744948635019786]
Спектр: [0.0058035107074719, -1.2684249019998715, -3.7947377036906516]
Последняя точка: [5.932838343634531, 0.8230925714341736, 0.4312110366183557]

---------
Индекс I0: 32; I0: -1.44813567839196
Индекс U0: 1; U0: 0.3
Начальные условия: [5.932838343634531, 0.8230925714341736, 0.4312110366183557]
Спектр: [0.004428971582371625, -1.2117185760823135, -3.833095738430337]
Последняя точка: [5.436706353673

Последняя точка: [8.32399345571772, 0.6600409372713064, 0.42189598136738643]

---------
Индекс I0: 57; I0: -1.4869547738693467
Индекс U0: 1; U0: 0.3
Начальные условия: [8.32399345571772, 0.6600409372713064, 0.42189598136738643]
Спектр: [0.00035427957309985445, -0.180153673247338, -4.443329057370536]
Последняя точка: [15.707211715556715, 0.5910547678030137, 0.43001452990863165]

---------
Индекс I0: 58; I0: -1.4885075376884422
Индекс U0: 1; U0: 0.3
Начальные условия: [15.707211715556715, 0.5910547678030137, 0.43001452990863165]
Спектр: [0.002598534494892866, -0.15094177013919588, -4.45839161911325]
Последняя точка: [4.442746568491537, 0.7762282561149821, 0.4205668637931735]

---------
Индекс I0: 59; I0: -1.4900603015075378
Индекс U0: 1; U0: 0.3
Начальные условия: [4.442746568491537, 0.7762282561149821, 0.4205668637931735]
Спектр: [0.004766064716306529, -0.12229619548704401, -4.471562206510292]
Последняя точка: [4.886487175167208, 0.7565074829543614, 0.4197363377765454]

---------
Индекс


---------
Индекс I0: 84; I0: -1.5288793969849246
Индекс U0: 1; U0: 0.3
Начальные условия: [8.593753049128663, 0.643399382437408, 0.427903172547612]
Спектр: [0.006661872866890747, -2.1774316335555226, -2.1812559728436898]
Последняя точка: [11.012104428766746, 0.6361108673380287, 0.42296017590535]

---------
Индекс I0: 85; I0: -1.5304321608040201
Индекс U0: 1; U0: 0.3
Начальные условия: [11.012104428766746, 0.6361108673380287, 0.42296017590535]
Спектр: [0.005712535622908573, -2.169554079619082, -2.1776936883970985]
Последняя точка: [3.122088372751556, 0.8373822596163738, 0.42608632155741144]

---------
Индекс I0: 86; I0: -1.5319849246231156
Индекс U0: 1; U0: 0.3
Начальные условия: [3.122088372751556, 0.8373822596163738, 0.42608632155741144]
Спектр: [0.005556361972348437, -2.1679804039869244, -2.171597246436689]
Последняя точка: [3.059792112462884, 0.8491737880096386, 0.4288741842556868]

---------
Индекс I0: 87; I0: -1.533537688442211
Индекс U0: 1; U0: 0.3
Начальные условия: [3.05979211

Индекс U0: 1; U0: 0.3
Начальные условия: [9.532393229518101, 0.8175185444472427, 0.449428550266676]
Спектр: [0.004724590271609362, -0.22357926830102862, -3.8265085659296485]
Последняя точка: [20.04664771358637, 0.5714255655018876, 0.44124592756282727]

---------
Индекс I0: 112; I0: -1.572356783919598
Индекс U0: 1; U0: 0.3
Начальные условия: [20.04664771358637, 0.5714255655018876, 0.44124592756282727]
Спектр: [0.005580369017184766, -0.051705713131853515, -3.982437528590294]
Последняя точка: [3.685488997934148, 0.8734270058642096, 0.4429468369841058]

---------
Индекс I0: 113; I0: -1.5739095477386935
Индекс U0: 1; U0: 0.3
Начальные условия: [3.685488997934148, 0.8734270058642096, 0.4429468369841058]
Спектр: [0.005088355034480653, -0.28035815352446086, -3.7378550186418638]
Последняя точка: [9.539536669458249, 0.6673844628355876, 0.420849532760786]

---------
Индекс I0: 114; I0: -1.575462311557789
Индекс U0: 1; U0: 0.3
Начальные условия: [9.539536669458249, 0.6673844628355876, 0.4208495327

...

Последняя точка: [8.27809055332621, 0.6905291083760603, 0.44753859956348785]

---------
Индекс I0: 166; I0: -1.6562060301507537
Индекс U0: 1; U0: 0.3
Начальные условия: [8.27809055332621, 0.6905291083760603, 0.44753859956348785]
Спектр: [0.0021861254314831102, -0.7676105753628384, -4.963626232615312]
Последняя точка: [8.831121472203398, 0.7639857239127595, 0.4311919225750779]

---------
Индекс I0: 167; I0: -1.6577587939698493
Индекс U0: 1; U0: 0.3
Начальные условия: [8.831121472203398, 0.7639857239127595, 0.4311919225750779]
Спектр: [0.0029433783009960458, -0.7504076625698693, -4.988474030026517]
Последняя точка: [10.870888325673063, 0.6544298958492876, 0.42285641329477913]

---------
Индекс I0: 168; I0: -1.6593115577889448
Индекс U0: 1; U0: 0.3
Начальные условия: [10.870888325673063, 0.6544298958492876, 0.42285641329477913]
Спектр: [0.0028859912718040776, -0.7223025542978306, -5.019698151914398]
Последняя точка: [6.8626756993301585, 0.7793324107918204, 0.4291361041517922]

---------
И


---------
Индекс I0: 193; I0: -1.6981306532663316
Индекс U0: 1; U0: 0.3
Начальные условия: [1.4614409915680524, 0.8881582377550562, 0.43410588915690845]
Спектр: [0.0032965638268428847, -1.018965201222492, -3.822075658201504]
Последняя точка: [11.450499509476012, 0.6806230120276794, 0.4250871284509774]

---------
Индекс I0: 194; I0: -1.6996834170854271
Индекс U0: 1; U0: 0.3
Начальные условия: [11.450499509476012, 0.6806230120276794, 0.4250871284509774]
Спектр: [0.22121262169775965, -0.001725498654848881, -5.2253861290245185]
Последняя точка: [5.351322088301021, 0.7595492358526527, 0.4198354906801451]

---------
Индекс I0: 195; I0: -1.7012361809045227
Индекс U0: 1; U0: 0.3
Начальные условия: [5.351322088301021, 0.7595492358526527, 0.4198354906801451]
Спектр: [0.3790610003236481, -0.006149296178413429, -5.110590923494282]
Последняя точка: [8.553583097711913, 0.7126485197291221, 0.4433097819138093]

---------
Индекс I0: 196; I0: -1.7027889447236182
Индекс U0: 1; U0: 0.3
Начальные условия:

Спектр: [0.004687012041723945, -2.1526399989698852, -3.0893731258985375]
Последняя точка: [9.98590457560624, 0.6364581203600627, 0.4217244110469573]

---------
Индекс I0: 21; I0: -1.4310552763819095
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [9.98590457560624, 0.6364581203600627, 0.4217244110469573]
Спектр: [0.0036889917101370503, -2.0182147903847505, -3.205194580399359]
Последняя точка: [4.073323107214818, 0.8107307061063843, 0.4222551863519691]

---------
Индекс I0: 22; I0: -1.432608040201005
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [4.073323107214818, 0.8107307061063843, 0.4222551863519691]
Спектр: [0.0058141621847887705, -1.9131133534111142, -3.296790563018887]
Последняя точка: [4.227867651589544, 0.7928515706725281, 0.4199961131711439]

---------
Индекс I0: 23; I0: -1.4341608040201006
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [4.227867651589544, 0.7928515706725281, 0.4199961131711439]
Спектр: [0.0067546196535119545, -1.818884211091741, 

Спектр: [0.004079168886761929, -0.5804853297295056, -4.231991472242592]
Последняя точка: [17.974998873347307, 0.6053721599213147, 0.43199432100880775]

---------
Индекс I0: 47; I0: -1.471427135678392
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [17.974998873347307, 0.6053721599213147, 0.43199432100880775]
Спектр: [0.004635505769695651, -0.5399084511421365, -4.256676938024274]
Последняя точка: [10.426692405291059, 0.626914636126633, 0.42390194060004044]

---------
Индекс I0: 48; I0: -1.4729798994974874
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [10.426692405291059, 0.626914636126633, 0.42390194060004044]
Спектр: [0.0036886240049429377, -0.5004299875520886, -4.277895495708998]
Последняя точка: [16.66739452359502, 0.6945762931661043, 0.4363131664760012]

---------
Индекс I0: 49; I0: -1.474532663316583
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [16.66739452359502, 0.6945762931661043, 0.4363131664760012]
Спектр: [0.004729859880096145, -0.4676058490508

Спектр: [0.00373489430172723, -0.804411318068061, -3.65392973013614]
Последняя точка: [7.089243325805534, 0.7996547926035059, 0.4324535422821039]

---------
Индекс I0: 73; I0: -1.5117989949748745
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [7.089243325805534, 0.7996547926035059, 0.4324535422821039]
Спектр: [0.004118144746575734, -0.9444795271859661, -3.507382308628515]
Последняя точка: [18.834160387190416, 0.6778024618560498, 0.4415932703819602]

---------
Индекс I0: 74; I0: -1.5133517587939698
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [18.834160387190416, 0.6778024618560498, 0.4415932703819602]
Спектр: [0.005080102962485924, -1.114226807737794, -3.3317626053408422]
Последняя точка: [8.401142532615058, 0.786497752663976, 0.4333753811168593]

---------
Индекс I0: 75; I0: -1.5149045226130653
Индекс U0: 2; U0: 0.30085427135678394
Начальные условия: [8.401142532615058, 0.786497752663976, 0.4333753811168593]
Спектр: [0.004358069279200489, -1.3130001419965092, -3.

In [ ]:
lyapmin,lyapmax = minimum(Λs[:, :, 1]), maximum( Λs[:, :, 1])

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1], xlabel = L"I0",ylabel = L"U0", xlabelsize = 50, ylabelsize = 50,
            xticklabelsize = 35, yticklabelsize = 35,
            xgridvisible  = false, ygridvisible = false)

hm = heatmap!(ax1, I0range, U0range, Λs[:, :, 1], colormap = :seismic)
Colorbar(f[1, 2], hm)
f